PyTorch_MNIST_Network2_3layer_Sigmoid_CrossEntropy_3

In [1]:
#import sys
# sys.exit()

In [2]:
import torch
from torch.utils.data import Dataset, random_split   
# See https://clay-atlas.com/us/blog/2021/08/25/pytorch-en-random-split-data-set/
from torchvision import datasets       # Gets standard data sets from internet
from torchvision.transforms import ToTensor, Lambda 
# ToTensor() converts numpy array to a tensor
import matplotlib.pyplot as plt
training_data = datasets.MNIST(root = "data",train = True,download = True,\
                               transform = ToTensor())
training_data, validation_data = random_split(training_data,[50000,10000],
                                              generator=torch.Generator().manual_seed(42))
# random_split breaks up the 60000 samples into 50000 training samples and 10000 validation samples
test_data = datasets.MNIST(root = "data",train = False,download = True,transform = ToTensor())

In [3]:
print('type(datasets) =', type(datasets))
print('type(training_data) =', type(training_data))
print('len(training_data) =', len(training_data))
print('type(training_data[0]) =', type(training_data[0]))
print('type(training_data[1]) =', type(training_data[1]))
print('len(training_data[0]) =', len(training_data[0]))
print('training_data[0][0].shape = ', training_data[0][0].shape)
print('training_data[0][1] =', training_data[0][1])
img,label = training_data[0]
print('img.shape =', img.shape)
print('label =',label)
#print(img)

type(datasets) = <class 'module'>
type(training_data) = <class 'torch.utils.data.dataset.Subset'>
len(training_data) = 50000
type(training_data[0]) = <class 'tuple'>
type(training_data[1]) = <class 'tuple'>
len(training_data[0]) = 2
training_data[0][0].shape =  torch.Size([1, 28, 28])
training_data[0][1] = 6
img.shape = torch.Size([1, 28, 28])
label = 6


In [4]:
print('type(datasets) =', type(datasets))
print('type(validation_data) =', type(validation_data))
print('len(validation_data) =', len(validation_data))
print('type(validation_data[0]) =', type(validation_data[0]))
print('type(validation_data[1]) =', type(validation_data[1]))
print('len(validation_data[0]) =', len(validation_data[0]))
print('validation_data[0][0].shape = ', validation_data[0][0].shape)
print('validation_data[0][1] =', validation_data[0][1])
img,label = validation_data[0]
print('img.shape =', img.shape)
print('label =',label)

type(datasets) = <class 'module'>
type(validation_data) = <class 'torch.utils.data.dataset.Subset'>
len(validation_data) = 10000
type(validation_data[0]) = <class 'tuple'>
type(validation_data[1]) = <class 'tuple'>
len(validation_data[0]) = 2
validation_data[0][0].shape =  torch.Size([1, 28, 28])
validation_data[0][1] = 4
img.shape = torch.Size([1, 28, 28])
label = 4


In [5]:
from torch.utils.data import DataLoader
batch_size=20
train_dataloader = DataLoader(training_data, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=True)
validation_dataloader = DataLoader(validation_data, batch_size=batch_size, shuffle=True)

In [6]:
print(type(test_dataloader))
print('len(test_dataloader) =',len(test_dataloader))   # number of batches
print('len(test_dataloader.dataset) =', len(test_dataloader.dataset))
print('len(test_dataloader)*batch_size =',len(test_dataloader)*batch_size)
print()
print(type(train_dataloader))
print('len(train_dataloader) =',len(train_dataloader))   # number of batches
print('len(train_dataloader.dataset) =', len(train_dataloader.dataset))
print('len(train_data_loader)*batch_size =',len(train_dataloader)*batch_size) 
print()
print(type(validation_dataloader))
print('len(validation_dataloader) =',len(validation_dataloader))   # number of batches
print('len(validation_dataloader.dataset) =', len(validation_dataloader.dataset))
print('len(validation_dataloader)*batch_size =',len(validation_dataloader)*batch_size) 

<class 'torch.utils.data.dataloader.DataLoader'>
len(test_dataloader) = 500
len(test_dataloader.dataset) = 10000
len(test_dataloader)*batch_size = 10000

<class 'torch.utils.data.dataloader.DataLoader'>
len(train_dataloader) = 2500
len(train_dataloader.dataset) = 50000
len(train_data_loader)*batch_size = 50000

<class 'torch.utils.data.dataloader.DataLoader'>
len(validation_dataloader) = 500
len(validation_dataloader.dataset) = 10000
len(validation_dataloader)*batch_size = 10000


In [7]:
#train_dataloader is not subscriptable so train_dataloader[0] gives an error
batch_example = next(iter(train_dataloader))
print('len(batch_example) =',len(batch_example))
print('type(batch_example) =',type(batch_example))
print()
batch_images = batch_example[0]
print('len(batch_images) =',len(batch_images))
print('type(batch_images) =', type(batch_images))
batch_labels = batch_example[1]
print()
print('batch_labels =', batch_labels)
print('type(batch_labels) =', type(batch_labels))

len(batch_example) = 2
type(batch_example) = <class 'list'>

len(batch_images) = 20
type(batch_images) = <class 'torch.Tensor'>

batch_labels = tensor([1, 1, 4, 0, 0, 9, 9, 1, 2, 7, 7, 3, 0, 2, 1, 3, 5, 9, 4, 5])
type(batch_labels) = <class 'torch.Tensor'>


In [8]:
from torch import nn
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device '.format(device))

Using cpu device 


In [9]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork,self).__init__()
        self.flatten = nn.Flatten()
        self.linear_sigmoid_stack = nn.Sequential(
            nn.Linear(28*28,100),
            nn.Sigmoid(),
            nn.Linear(100,10),
        )
    def forward(self,x):
        x = self.flatten(x)
        logits = self.linear_sigmoid_stack(x)
        return logits


In [10]:
model =  NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_sigmoid_stack): Sequential(
    (0): Linear(in_features=784, out_features=100, bias=True)
    (1): Sigmoid()
    (2): Linear(in_features=100, out_features=10, bias=True)
  )
)


In [11]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X,y) in enumerate(dataloader):
        X = X.to(device)
        y = y.to(device)
        pred = model(X)
        #print(pred.shape)
        #sys.exit()
        loss = loss_fn(pred,y)
    
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
#         if batch %100 == 0:
#             loss, current = loss.item(), batch*len(X)
#             print(f"loss: {loss: > 7f} [{current: > 5d}/{size:>5d}]")

In [12]:
def test_loop(dataloader, model, loss_fn, valid_or_test_flag = "valid"):
    size = len(dataloader.dataset)
    test_loss, correct = 0,0
    with torch.no_grad():
        for X,y in dataloader:
            X = X.to(device)
            y = y.to(device)
            pred = model(X)
#             print(pred.shape)
#             sys.exit()
            test_loss += loss_fn(pred,y).item()
            correct += (pred.argmax(1)==y).type(torch.float).sum().item()
    test_loss /= size
    correct /= size
    if valid_or_test_flag == "test":
        print(f"Test Error: \n Accuracy {(100*correct):>0.1f}%, Avg. loss: {test_loss:>8f} \n")
    else: 
        print(f"Validation Error: \n Accuracy {(100*correct):>0.1f}%, Avg. loss: {test_loss:>8f} \n")

In [13]:
learning_rate = 0.1
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),lr = learning_rate)
epochs = 60
for t in range(epochs):
    print(f"Epoch {t+1})\n --------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(validation_dataloader, model, loss_fn,valid_or_test_flag = "valid")
print("Done Training")
print()
print("Begin Test")
test_loop(test_dataloader, model, loss_fn,valid_or_test_flag = "test")

Epoch 1)
 --------------------------


Validation Error: 
 Accuracy 90.8%, Avg. loss: 0.015961 

Epoch 2)
 --------------------------
Validation Error: 
 Accuracy 92.2%, Avg. loss: 0.013277 

Epoch 3)
 --------------------------
Validation Error: 
 Accuracy 93.5%, Avg. loss: 0.011359 

Epoch 4)
 --------------------------
Validation Error: 
 Accuracy 93.8%, Avg. loss: 0.010275 

Epoch 5)
 --------------------------
Validation Error: 
 Accuracy 94.5%, Avg. loss: 0.009282 

Epoch 6)
 --------------------------
Validation Error: 
 Accuracy 94.8%, Avg. loss: 0.008461 

Epoch 7)
 --------------------------
Validation Error: 
 Accuracy 95.2%, Avg. loss: 0.007975 

Epoch 8)
 --------------------------
Validation Error: 
 Accuracy 95.5%, Avg. loss: 0.007440 

Epoch 9)
 --------------------------
Validation Error: 
 Accuracy 96.0%, Avg. loss: 0.007043 

Epoch 10)
 --------------------------
Validation Error: 
 Accuracy 96.0%, Avg. loss: 0.006600 

Epoch 11)
 --------------------------
Validation Error: 
 Accuracy 96.3%, Avg. loss: 0

In [14]:
target_transform = Lambda(lambda y: torch.zeros(
    10, dtype=torch.float).scatter_(dim=0, index=torch.tensor(y), value=1))